## Laboratorium 3

### Opis
Celem laboratorium jest klasyfikacja liści za pomocą metody *k* najbliższych sąsiadów - *k*NN (*k Nearest Neighbors*) z wykorzystaniem normalizacji.


### Zbiór danych

Zbiór danych znajduje się w katalogu `dataset/*`. Jest to zmodyfikowany zbiór danych znajdujący się pod adresem: <https://archive.ics.uci.edu/ml/datasets/leaf>.

### Przesyłanie zadań

Wszystkie pliki należy spakować archiwizatorem **zip** i przesłać za pośrednictwem platformy WIKAMP. Poniżej oczekiwana zawartość archiwum:

```
+-- 📂 [IMIE.NAZWISKO].zip
    +-- 📜 Lab03.ipynb
    +-- 📂 dataset
        +-- 📜 dataset.npz
        +-- 📜 ReadMe.pdf
```



### Zadanie 1

* Wybierz 10 dowolnych gatunków liści (będziesz je używać w kolejnych zadaniach) oraz wszystkie cechy.
* Przeprowadź klasyfikację za pomocą klasyfikatora [*k*NN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).
* Znajdź optymalne *k*.
* Wyświetl najwyższy wynik klasyfikacji (*accuracy*).


In [9]:
import numpy as np
import matplotlib.pyplot as plt

#Loading data
with open('./dataset/dataset.npz', 'rb') as f:
    data = np.load(f)
    train, test = data['train'], data['test']

chosen_leafs = [1,2,3,4,5,6,7,8,9,10]

binary_vector_train = np.isin(train[:,0], chosen_leafs)
binary_vector_test = np.isin(test[:,0], chosen_leafs)

train_new = train[binary_vector_train]
test_new = test[binary_vector_test]

"""
for i in range(1,11):
    class_size = len(train_new[train_new[:,0]==i])
    print(f"Liczba probek klasy {i} wynosi: {class_size}")
"""
print("Zbior treningowy:")
#Dla treningowego
y_train = train_new[:,0].copy()
x_train = train_new[:,2:].copy()
print("Liczba probek: ",len(y_train))
print("Liczba klas: :", len(np.unique(y_train)))
print("Liczba cech: ",len(x_train[0]))

print("\nZbior testowy:")
#Dla testwowego
y_test = test_new[:,0].copy()
x_test = test_new[:,2:].copy()
print("Liczba probek: ",len(y_test))
print("Liczba klas: :", len(np.unique(y_test)))
print("Liczba cech: ",len(x_test[0]))



Zbior treningowy:
Liczba probek:  671
Liczba klas: : 10
Liczba cech:  14

Zbior testowy:
Liczba probek:  517
Liczba klas: : 10
Liczba cech:  14


In [10]:
#Klasyfikacja KNN
import sklearn as skl
#Podzial probek na treningowe i testowe za pomoca funkcji scikit train_test_split
from sklearn.model_selection import train_test_split
#The same random state for the repeatability of scores
random_s = 15
#Train
y_train = train_new[:,0].copy()
x_train = train_new[:,2:].copy()

#Test
y_test = test_new[:,0].copy()
x_test = test_new[:,2:].copy()


#Zaimportowanie i dopasowanie klasyfikatora przykladowo na 4 najblizszych sasiadach
from sklearn.neighbors import KNeighborsClassifier
K_nearest = KNeighborsClassifier(n_neighbors=3)
K_nearest.fit(x_train, y_train)
#Predykcja na podstawie danych testowych
y_pred = K_nearest.predict(x_test)
acc = skl.metrics.accuracy_score(y_test, y_pred)
print("Accuracy for chosen k nearest neighbours (13): ",round(acc,4))

def check_best_neighbours_n(x_train,y_train,x_test,y_test):
    best_acc = 0
    best_k = None
    for k in range(1,len(x_train)):
        K_nearest = KNeighborsClassifier(n_neighbors=k)
        K_nearest.fit(x_train, y_train)
        y_pred = K_nearest.predict(x_test)
        acc = skl.metrics.accuracy_score(y_test, y_pred)

        if acc> best_acc:
            best_acc = acc
            best_k = k

    return best_acc, best_k

acc, k = check_best_neighbours_n(x_train,y_train,x_test,y_test)

print("Best acc: ",round(acc,4))
print("Best k neighbours: ",k)




Accuracy for chosen k nearest neighbours (13):  0.7369
Best acc:  0.7524
Best k neighbours:  9


### Zadanie 2
* Znormalizuj dane w zakresie 0-1. Możesz do tego celu wykorzystać gotową funkcję z biblioteki scikit-learn https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html.
>    **Ważne: współczynniki powinny być obliczone na zbiorze treningowym i te same powinny zostać zastosowane do normalizacji zbioru testowego.**
* Ponownie wykonaj klasyfikację z tymi samymi parametrami (co w zadaniu 1), ale na danych znormalizowanych.
* Znajdź optymalne *k*.
* Wyświetl wynik w postaci wykresu, gdzie na osi X znajduje się *k*, a na osi Y *accuracy*.
* Wyświetl najwyższy wynik klasyfikacji (*accuracy*).


In [11]:
#Zrobic minmaxskaler z scikit learn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#Ponowne wywołanie danych w celu mozliwosci ponawiania komorki (cell) wielokrotnie
#Train
y_train = train_new[:,0].copy()
x_train = train_new[:,2:].copy()

#Test
y_test = test_new[:,0].copy()
x_test = test_new[:,2:].copy()

#Fit na danych treningowych
scaler.fit(x_train)
x_train_normalized_skl = scaler.transform(x_train)
x_test_normalized_skl = scaler.transform(x_test)

print("Scikit normalization")
print("Min x_train: ",x_train_normalized_skl.min())
print("Max x_train: ",x_train_normalized_skl.max())
print("Min x_test: ",x_test_normalized_skl.min())
print("Max x_test: ",x_test_normalized_skl.max())

#Test for normalization by scikitlearn
acc_scikit, k_scikit = check_best_neighbours_n(x_train_normalized_skl, y_train, x_test_normalized_skl, y_test)

print("Scikit_norm best acc: ",acc_scikit)
print("Scikit best k: ", k_scikit)

#Zrobic wlasna normalizacja min max
def own_min_max(x_train, x_test):
    #Max min per feature (along axis 0 )
    x_min = x_train.min(axis=0)
    x_max = x_train.max(axis=0)
    
    #Zakres wartosci
    range_values = x_max - x_min
    #Zeby nie bylo errora /0
    range_values[range_values == 0] = 1
    #Normalizacja
    x_train_norm = (x_train - x_min) / range_values
    x_test_norm = (x_test - x_min) / range_values
    
    return x_train_norm, x_test_norm
    

x_train_normalized_own,x_test_normalized_own = own_min_max(x_train,x_test)
print("###############################################")
print("Own MinMax normalization")
print("Min x_train: ",x_train_normalized_own.min())
print("Max x_train: ",x_train_normalized_own.max())
print("Min x_test: ",x_test_normalized_own.min())
print("Max x_test: ",x_test_normalized_own.max())

acc_own, k_own = check_best_neighbours_n(x_train_normalized_own, y_train, x_test_normalized_own, y_test)
print("Own_norm best acc: ",acc_scikit)
print("Own_norm best k: ", k_scikit)



Scikit normalization
Min x_train:  0.0
Max x_train:  1.0000000000000002
Min x_test:  -0.15618629851288823
Max x_test:  1.087568483623526
Scikit_norm best acc:  0.816247582205029
Scikit best k:  14
###############################################
Own MinMax normalization
Min x_train:  0.0
Max x_train:  1.0
Min x_test:  -0.15618629851288823
Max x_test:  1.0875684836235262
Own_norm best acc:  0.816247582205029
Own_norm best k:  14


### Zadanie 3

Napisz wnioski

In [ ]:
"""
Normalizacja powoduje zachowanie relacji w obrębie jednej cechy, jendocześnie sprowadzając różne cechy do tego samego zakresu,
co pozwala na ich efektywne porównywanie, oraz wyrównanie wpływu cech na wynik gdyby miałe one różne rzędy wielkości.
W przeciwnym wypadku algorytm (szczególnie tak podstawowy jak K-nearest neighbours) odczytałby cechę o większej rozpiętości jako mniej istotną, 
ponieważ cecha znajdowała się dalej niż cecha o mniejszej rozpiętości, mimo że w istocie powinny mieć one ten sam wpływ na predykcje
"""